In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(False) / "coeditor-large-query-basic/checkpoint-221507"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
service = EditPredictionService(
    proj_root(),
    model,
    encoder=BasicQueryEditEncoder(max_ref_tks=512),
    batch_args=BatchArgs(
        max_ref_tks=50 * 512, max_ref_dropout=0.0, shuffle_extra_ids=False
    ),
)


/home/jiayi/Projects/SPOT/src/coeditor/retrieval_model.py:255: UserWarning: No extra args found, using default model setting.
  warnings.warn("No extra args found, using default model setting.")


In [6]:
service.suggest_edit(Path("src/coeditor/history.py"), 185)
# service.suggest_edit(Path("src/spot/utils.py"), (218, 0))

num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
len(input_tks) = 244
len(references) = 4
========== Predicted code change ==========
* Modified: 
    ... 11 lines omitted ...
                        code = f.header_body_code[1]
                    else:
                        code = f.code
                    return normalize_code_by_ast(code)
 
                return {
    -               k: cast(Modified[PythonFunction], change)
    +               k: normalize_code_by_ast(cast(Modified[PythonFunction], change))
                    for k, change in self.modified.items()
                    if isinstance(change.before, PythonFunction)
                    and isinstance(change.after, PythonFunction)
                    and (
                        not ast_must_change
                        or norm_code(change.before)!= norm_code(change.after)
    ... 2 lines omitted ...


In [4]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
1,construct project edit,1,1.512312,1.512312
3,run model,1,0.584002,0.584002
0,get target element,1,0.114759,0.114759
2,encode edits,1,0.045101,0.045101
